In [4]:
import tensorflow as tf
from tensorflow import keras
from component.respiratory_tract import get_respiratory_tract_model
from utils.feature_extraction import  get_MFCCS_v2
from sklearn.utils import class_weight
import numpy as np
import glob
import tqdm
from imblearn.over_sampling import SMOTE

In [3]:
n_mfccs = 39
num_segments = 70
segment_length = 1024

In [ ]:
X1 = glob.glob("coswara_wavs/train/p/*.wav")
X0 = glob.glob("coswara_wavs/train/n/*.wav")


In [6]:
y = []
X = []
for file in tqdm.tqdm(X1):
    try:
        X.append(get_MFCCS_v2(file, n_mfccs=n_mfccs, num_segments=num_segments, segment_length=segment_length))
        y.append(1)
    except:
        pass
for file in tqdm.tqdm(X0):
    try:
        X.append(get_MFCCS_v2(file, n_mfccs=n_mfccs, num_segments=num_segments, segment_length=segment_length))
        y.append(0)
    except:
        pass

  0%|          | 0/296 [00:00<?, ?it/s]d:\project\papyrus\venv\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1024
  n_fft, y.shape[-1]
100%|██████████| 2088/2088 [14:34<00:00,  2.39it/s]


In [10]:
X = np.array(X)
y = np.array(y)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)

ValueError: Found array with dim 4. Estimator expected <= 2.

In [3]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size, n_mfcs, num_segment, segment_len):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indexes = set(np.arange(len(X)))
        self.num_segments = num_segment
        self.n_mfccs = n_mfcs
        self.segment_length = segment_len

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = random.sample(self.indexes, self.batch_size)
        X = []
        y = []
        for index in indexes:
            try:
                X.append(get_MFCCS_v2(self.X[index], n_mfccs=self.n_mfccs, num_segments=self.num_segments, segment_length=self.segment_length))
                y.append(self.y[index])
            except:
                continue
        return np.array(X), np.array(y, dtype=int)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y),
                                                 y)
class_weights = {1 - i: x for i,x in enumerate(class_weights)}
print(f"Class weights: {class_weights}")

Class weights: {1: 0.9391691394658753, 0: 1.0692567567567568}


d:\project\papyrus\venv\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1], y=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [6]:
train_generator = DataGenerator(X_train, y_train, 8, n_mfccs, num_segments, segment_length)
val_generator = DataGenerator(X_val, y_val, 8, n_mfccs, num_segments, segment_length)

In [7]:
model = get_respiratory_tract_model(input_shape=(n_mfccs*3+3,num_segments,1))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])

In [8]:
history = model.fit(train_generator, steps_per_epoch=20, validation_steps=20, epochs=200, validation_data=val_generator, class_weight=class_weights,
                    callbacks=[tf.keras.callbacks.ModelCheckpoint('pretrained_weight/model_new_paper.h5', monitor="val_acc", save_weights_only=True, verbose=1, mode='max', save_best_only=True),
                             tf.keras.callbacks.TensorBoard(log_dir='logs')])

d:\project\papyrus\venv\lib\site-packages\librosa\core\spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1024
  n_fft, y.shape[-1]


Epoch 1/200
20/20 [==============================] - 103s 5s/step - loss: 0.7052 - acc: 0.4719 - val_loss: 0.8172 - val_acc: 0.4875

In [9]:
from sklearn.metrics import classification_report
import tqdm
X1_test = glob.glob("coswara_wavs/test/p/*.wav")
X0_test = glob.glob("coswara_wavs/test/n/*.wav")
X_test = []
y_test = []
for audio_path in tqdm.tqdm(X1_test):
    try:
        X_test.append(get_MFCCS_v2(audio_path, n_mfccs=n_mfccs, num_segments=num_segments))
        y_test.append(1)
    except:
        continue

for audio_path in tqdm.tqdm(X0_test):
    try:
        X_test.append(get_MFCCS_v2(audio_path, n_mfccs=n_mfccs, num_segments=num_segments))
        y_test.append(0)
    except:
        continue
X_test = np.array(X_test)

100%|██████████| 514/514 [03:38<00:00,  2.35it/s]


In [10]:
model.load_weights("pretrained_weight/model_new_paper.h5")
y_pred = model.predict(X_test)
y_pred_r = np.rint(y_pred)
print(classification_report(y_pred_r, y_test))

              precision    recall  f1-score   support

         0.0       0.71      0.90      0.80       400
         1.0       0.47      0.20      0.28       179

    accuracy                           0.68       579
   macro avg       0.59      0.55      0.54       579
weighted avg       0.64      0.68      0.64       579



In [10]:
model.load_weights("pretrained_weight/model_new_paper.h5")
y_pred = model.predict(X_test)
y_pred_r = np.rint(y_pred)
print(classification_report(y_pred_r, y_test))


              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93       579
         1.0       0.00      0.00      0.00         0

    accuracy                           0.87       579
   macro avg       0.50      0.44      0.47       579
weighted avg       1.00      0.87      0.93       579



d:\project\papyrus\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\project\papyrus\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\project\papyrus\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
